In [1]:
import numpy as np
import plotly.express as px
from instance_mongodb import instance_mongodb_sei

from pymatgen.core.structure import Molecule
from pymatgen.analysis.local_env import OpenBabelNN
from pymatgen.analysis.graphs import MoleculeGraph

from monty.serialization import loadfn, dumpfn

import torch

from e3nn import o3

from minimal_basis.data.data_reaction import ModifiedCoefficientMatrix

from utils import rotate_three_dimensions, subdiagonalize_matrix

In [2]:
def add_bounds_to_figure(fig):

    fig.add_shape(
        type="line",
        x0=-0.5,
        y0=4.5,
        x1=6.5,
        y1=4.5,
        line=dict(color="black", width=2),
    )
    fig.add_shape(
        type="line",
        x0=-0.5,
        y0=5.5,
        x1=6.5,
        y1=5.5,
        line=dict(color="black", width=2),
    )
    fig.add_shape(
        type="line",
        x0=-0.5,
        y0=-0.5,
        x1=-0.5,
        y1=6.5,
        line=dict(color="black", width=2),
    )
    fig.add_shape(
        type="line",
        x0=4.5,
        y0=-0.5,
        x1=4.5,
        y1=6.5,
        line=dict(color="black", width=2),
    )

    fig.update_xaxes(
        ticktext=[ "O1s", "O2s", "O2p", "O2p", "O2p", "H1s", "H1s",],
        tickvals=np.arange(11),
    )
    fig.update_yaxes(
        ticktext=["O1s", "O2s", "O2p", "O2p", "O2p", "H1s", "H1s"],
        tickvals=np.arange(11),
    )

    return fig

In [3]:
db = instance_mongodb_sei(project="mlts")
collection = db.rotated_waters_dataset

hamiltonians = []
angles = []
structures = []
overlap_matrices = []
coefficients = []
eigenvalues = []

for doc in collection.find({}).limit(10):
    hamiltonians.append(doc["fock_matrices"][0])
    overlap_matrices.append(doc["overlap_matrices"][0])
    structure = Molecule.from_dict(doc["structures"][0])
    structures.append(structure)
    coefficients.append(doc["coeff_matrices"][0])
    angles.append(doc["angles"])
    eigenvalues.append(doc["eigenvalues"][0])

hamiltonians = np.array(hamiltonians)
overlap_matrices = np.array(overlap_matrices)
coefficients = np.array(coefficients)
angles = np.array(angles)
eigenvalues = np.array(eigenvalues)
eigenvalues = eigenvalues[..., np.newaxis, :]

In [4]:
# Analyse one molecule's input to the CoefficientMatrix
alpha_coeff_matrix = coefficients[0,0]
chosen_eigenstate = 3
molecule_graph = MoleculeGraph.with_local_env_strategy(structures[0], OpenBabelNN())
coeff_matrix = ModifiedCoefficientMatrix(
    molecule_graph=molecule_graph,
    basis_info_raw=loadfn("input_files/sto-3g.json"),
    coefficient_matrix=alpha_coeff_matrix,
    store_idx_only=chosen_eigenstate,
    set_to_absolute=True,
)

In [5]:
# Plot the coefficient matrix as it is
padded_coeff_matrix = coeff_matrix.get_padded_representation()
fig = px.imshow(coeff_matrix.get_padded_representation(), animation_frame=0)

fig.update_layout(title_text='DFT computed coefficient matrix for a rotated water molecule', title_x=0.5)
fig.update_xaxes(title_text="Eigenvalues index")
fig.update_xaxes(
    ticktext=np.arange(8),
    tickvals=np.arange(8),
)
fig.update_yaxes(
    ticktext=["s", "s", "p", "p", "p"],
    tickvals=np.arange(11),
)
fig.show()

In [ ]:
# Plot the padded coefficient matrix for each atom
all_padded_coeff_matrices = []
for atom_idx, atom in enumerate(coeff_matrix.molecule_graph.molecule):
    padded_coeff_matrix = coeff_matrix.get_minimal_basis_representation_atom(atom_idx)
    all_padded_coeff_matrices.append(padded_coeff_matrix)
all_padded_coeff_matrices = np.array(all_padded_coeff_matrices)

fig = px.imshow(all_padded_coeff_matrices, animation_frame=0) 
fig.update_yaxes(
    ticktext=["s", "p", "p", "p"],
    tickvals=np.arange(4),
)
fig.update_xaxes(
    ticktext=[],
    tickvals=[],
)
fig.show()